# Chapter 14. Recurrent Neural Network
  * prediction on data sequence of time series

## Recurrent Neurons
  * Unlike neurons in feed forward network, ouput can be fed into itself
  * not only backpropagation, but also **unrolling** is required during training
  * Memory Cells
    * output from itself of previous time step is called **cell's state**

## Input & Output
  * Sequence to Sequence 
    * e.g. for given time serise input, time series with future prediction are returned 
  * Sequence to Vector (e.g. sentiment analysis) 
  * Vector to Sequence (e.g. captioning image)
  * Encoder - Decoder (e.g. translator) (Seq2Vec - Vec2Seq)


## Recurrent Neuron model

In [49]:
# Basic Model of Recurrent Neuron
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X0, Wx) + b) 
'''
Y2 
Y3
.
.

''' 





[array([[-0.42141908, -0.809767  , -0.14516923,  0.73781574, -0.06028287],
       [ 0.30095518, -0.97897786,  0.07166494,  0.99962914, -0.32041547],
       [ 0.7896704 , -0.99785525,  0.28193057,  0.9999995 , -0.5397935 ],
       [ 0.99673885,  0.5778276 ,  0.67889106,  0.99997693,  0.8691432 ]],
      dtype=float32), array([[ 0.98770326, -0.9142149 ,  0.46105415,  1.        , -0.6597372 ],
       [-0.7081102 ,  0.8166296 , -0.24081503,  0.7314205 , -0.28680807],
       [ 0.6079217 ,  0.08461839,  0.3418136 ,  0.9999945 , -0.7518167 ],
       [ 0.43567273,  0.36083198,  0.8883119 ,  0.9894406 ,  0.1835967 ]],
      dtype=float32)]


## RNN network in simplest form

In [50]:

import tensorflow as tf
import numpy as np

tf.reset_default_graph()

n_inputs = 3
n_neurons = 5
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

Y0, Y1 = output_seqs
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run([Y0,Y1],feed_dict={X0:X0_batch, X1:X1_batch}))
    

[array([[-0.7027918 ,  0.6674094 ,  0.00498064, -0.43444335,  0.6118566 ],
       [-0.99949366,  0.917452  ,  0.98799443, -0.9984121 ,  0.97833204],
       [-0.9999993 ,  0.98158765,  0.9999263 , -0.9999968 ,  0.9990042 ],
       [-0.9997933 ,  0.9314043 ,  0.99996305, -0.99975854,  0.13153642]],
      dtype=float32), array([[-0.99999994,  0.7348813 ,  0.9999998 , -1.        ,  0.9976633 ],
       [ 0.31974855, -0.92133725,  0.25694522, -0.1987123 , -0.8016095 ],
       [-0.9999525 , -0.7069174 ,  0.99995714, -0.99998647,  0.84029955],
       [-0.9760914 , -0.9519193 ,  0.99266493, -0.9831109 ,  0.13102499]],
      dtype=float32)]


## Static Unrolling RNN
- **static_rnn()** -> build unrolled RNN network by chaining RNN cells
- each instance of input data is a sequence..
  * thus, rank of input is 3 (3D) => (batch, steps, inputs)
  * but data should be fed list of steps
    1. transpose(X, perm=[1,0,2]) -> (steps, batch, inputs)
    2. unstack(result from 1) -> (?, inputs)
    * conversion is complex !!!
- static unrolling is memory consuming 
  * each optimization iteration, all tensor values should be stored (because it's recurrent neuron!!)

In [47]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

n_steps = 10
n_neurons = 10
n_inputs = 2
n_batch = 10

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.transpose(X, perm=[1,0,2])
print(X_seqs.get_shape())
X_seqs_ustk = tf.unstack(X_seqs)
print(X_seqs_ustk[0].get_shape())

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs_ustk, dtype=tf.float32)

outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])
x_batch = np.array([[[i for i in range(o, o + n_inputs)] for o in range(n_steps)] for _ in range(n_batch)])
print(x_batch[0])
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print('Input')
    print(sess.run(X, feed_dict={X: [x_batch[0]]}))
    print('After Transpose')
    seqs = sess.run(X_seqs, feed_dict={X: [x_batch[0]]})
    print(type(seqs))
    for l in seqs:
        print(l)
    print('After Unstack')
    seqs_ustk = sess.run(X_seqs_ustk, feed_dict={X: [x_batch[0]]})
    print(type(seqs_ustk))
    for l in seqs_ustk:
        print(l)
    print('Final Data')
    print(sess.run(outputs, feed_dict={X: [x_batch[0]]}))



(10, ?, 2)
(?, 2)
[[ 0  1]
 [ 1  2]
 [ 2  3]
 [ 3  4]
 [ 4  5]
 [ 5  6]
 [ 6  7]
 [ 7  8]
 [ 8  9]
 [ 9 10]]
Input
[[[ 0.  1.]
  [ 1.  2.]
  [ 2.  3.]
  [ 3.  4.]
  [ 4.  5.]
  [ 5.  6.]
  [ 6.  7.]
  [ 7.  8.]
  [ 8.  9.]
  [ 9. 10.]]]
After Transpose
<class 'numpy.ndarray'>
[[0. 1.]]
[[1. 2.]]
[[2. 3.]]
[[3. 4.]]
[[4. 5.]]
[[5. 6.]]
[[6. 7.]]
[[7. 8.]]
[[8. 9.]]
[[ 9. 10.]]
After Unstack
<class 'list'>
[[0. 1.]]
[[1. 2.]]
[[2. 3.]]
[[3. 4.]]
[[4. 5.]]
[[5. 6.]]
[[6. 7.]]
[[7. 8.]]
[[8. 9.]]
[[ 9. 10.]]
Final Data
[[[-0.39882165  0.3542116  -0.07195529 -0.36859003  0.07567247
   -0.02040198  0.46789715  0.2543415   0.23556472 -0.18304275]
  [-0.4633027   0.3413062  -0.7954224  -0.53963345  0.18950076
    0.09189894  0.5788722   0.37592298  0.41902596 -0.14458562]
  [-0.7979515   0.3925388  -0.94819856 -0.7438859   0.53125167
    0.37850165  0.7288936   0.796322    0.78594965 -0.06165339]
  [-0.9072312  -0.03132254 -0.9926383  -0.7012744   0.7915995
    0.17497925  0.8695304   0.901851

## Dynamic Unrolling
 * support swapping memory contents CPU <-> GPU
   * prevent OOM
   * no need for tensor conversion 


In [48]:
import tensorflow as tf
tf.reset_default_graph()

n_steps = 10
n_neurons = 10
n_inputs = 20
n_batch = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
x_batch = np.array([[[i for i in range(o, o + n_inputs)] for o in range(n_steps)] for _ in range(n_batch)])

basic_cell = tf.contrib.rnn.BasicRNNCell(n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(outputs, feed_dict={X:x_batch}).shape)
    print(sess.run(states, feed_dict={X:x_batch}).shape)
    

(1, 10, 10)
(1, 10)
